# Chi Shift Physics Demonstration

This notebook demonstrates the physics-based chi shift calculation for transmon qubits in the dispersive regime. We'll explore how anharmonicity and coupling strength affect the dispersive shifts and compare with analytical approximations.

## Overview

In dispersive readout, a transmon qubit is coupled to a resonator with coupling strength $g$ much smaller than their frequency detuning $\Delta = \omega_r - \omega_q$. This leads to state-dependent frequency shifts (chi shifts) of the resonator:

$$\chi_n = \sum_m \frac{|g_{nm}|^2}{\omega_r - \omega_{nm}}$$

where $g_{nm}$ are the coupling matrix elements and $\omega_{nm} = E_m - E_n$ are transition frequencies.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from leeq.theory.simulation.numpy.dispersive_readout.physics import ChiShiftCalculator
from leeq.theory.simulation.numpy.dispersive_readout.transmon_physics import (
    calculate_transmon_energies,
    calculate_transition_frequencies,
    calculate_coupling_matrix_elements
)

# Set up matplotlib for nice plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Basic Chi Shift Calculation

Let's start with a typical transmon configuration and calculate the chi shifts for the first few energy levels.

In [ ]:
# Typical transmon parameters
f_r = 7000          # Resonator frequency (MHz)
f_q = 5000          # Qubit frequency (MHz)
anharmonicity = -250 # Anharmonicity (MHz)
g = 100             # Coupling strength (MHz)
num_levels = 4      # Number of levels to include

# Initialize calculator
calc = ChiShiftCalculator()

# Calculate chi shifts
chi_shifts = calc.calculate_chi_shifts(
    f_r=f_r,
    f_q=f_q,
    anharmonicity=anharmonicity,
    g=g,
    num_levels=num_levels
)

print("System Parameters:")
print(f"Resonator frequency: {f_r} MHz")
print(f"Qubit frequency: {f_q} MHz")
print(f"Detuning: {f_r - f_q} MHz")
print(f"Anharmonicity: {anharmonicity} MHz")
print(f"Coupling strength: {g} MHz")
print(f"g/Δ ratio: {g / abs(f_r - f_q):.3f}")
print()
print("Chi shifts (MHz):")
for i, chi in enumerate(chi_shifts):
    print(f"χ_{i} = {chi:.3f}")

## 2. Comparison with Two-Level Approximation

Let's compare the multi-level calculation with the simple two-level approximation: $\chi = g^2/\Delta$.

In [ ]:
# Two-level approximation
chi_2level = calc.calculate_two_level_chi(f_r, f_q, g)
chi_2level_diff = chi_shifts[1] - chi_shifts[0]

print("Two-level vs Multi-level Comparison:")
print(f"Two-level formula: χ = g²/Δ = {chi_2level:.3f} MHz")
print(f"Multi-level result: χ₁ - χ₀ = {chi_2level_diff:.3f} MHz")
print(f"Relative difference: {abs(chi_2level - chi_2level_diff) / chi_2level * 100:.1f}%")
print()
print("Level-dependent chi differences:")
for i in range(1, len(chi_shifts)):
    chi_diff = chi_shifts[i] - chi_shifts[i-1]
    enhancement = chi_diff / chi_2level if chi_2level != 0 else 0
    print(f"χ_{i} - χ_{i-1} = {chi_diff:.3f} MHz (factor {enhancement:.2f} vs 2-level)")

## 3. Effect of Anharmonicity

The anharmonicity controls the spacing between energy levels. Let's see how different anharmonicity values affect the chi shifts.

In [ ]:
# Range of anharmonicity values
alpha_values = np.linspace(-400, -100, 7)  # From -400 to -100 MHz

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Calculate chi shifts for different anharmonicities
chi_results = []
for alpha in alpha_values:
    chi = calc.calculate_chi_shifts(f_r, f_q, alpha, g, num_levels)
    chi_results.append(chi)
    
chi_results = np.array(chi_results)

# Plot chi shifts vs level number
colors = plt.cm.viridis(np.linspace(0, 1, len(alpha_values)))
for i, (alpha, chi, color) in enumerate(zip(alpha_values, chi_results, colors)):
    ax1.plot(range(num_levels), chi, 'o-', color=color, label=f'α = {alpha:.0f} MHz')

ax1.set_xlabel('Transmon Level')
ax1.set_ylabel('Chi Shift (MHz)')
ax1.set_title('Chi Shifts vs Anharmonicity')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot chi differences vs anharmonicity
chi_01 = chi_results[:, 1] - chi_results[:, 0]
chi_12 = chi_results[:, 2] - chi_results[:, 1]
chi_23 = chi_results[:, 3] - chi_results[:, 2]

ax2.plot(alpha_values, chi_01, 'o-', label='χ₁ - χ₀', linewidth=2)
ax2.plot(alpha_values, chi_12, 's-', label='χ₂ - χ₁', linewidth=2)
ax2.plot(alpha_values, chi_23, '^-', label='χ₃ - χ₂', linewidth=2)

ax2.set_xlabel('Anharmonicity (MHz)')
ax2.set_ylabel('Chi Difference (MHz)')
ax2.set_title('Level-to-Level Chi Differences')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Observations:")
print("• More negative anharmonicity → larger level spacing → smaller chi for higher levels")
print("• Chi differences decrease as anharmonicity becomes more negative")
print("• Higher levels show stronger anharmonicity dependence")

## 4. Effect of Coupling Strength

The coupling strength $g$ controls the overall scale of dispersive shifts. Let's explore how chi shifts scale with coupling.

In [ ]:
# Range of coupling strengths
g_values = np.linspace(50, 200, 6)  # From 50 to 200 MHz

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Calculate chi shifts for different coupling strengths
chi_vs_g = []
dispersive_valid = []

for g_val in g_values:
    chi = calc.calculate_chi_shifts(f_r, f_q, anharmonicity, g_val, num_levels)
    chi_vs_g.append(chi)
    # Check dispersive regime validity
    is_dispersive = calc.validate_dispersive_regime(f_r, f_q, g_val)
    dispersive_valid.append(is_dispersive)
    
chi_vs_g = np.array(chi_vs_g)

# Plot chi shifts vs coupling strength
colors = plt.cm.plasma(np.linspace(0, 1, num_levels))
for level in range(num_levels):
    marker = 'o' if all(dispersive_valid) else ('o' if level < 2 else 's')
    ax1.plot(g_values, chi_vs_g[:, level], marker+'-', color=colors[level], 
             label=f'χ_{level}', linewidth=2)

ax1.set_xlabel('Coupling Strength g (MHz)')
ax1.set_ylabel('Chi Shift (MHz)')
ax1.set_title('Chi Shifts vs Coupling Strength')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add dispersive regime boundary
g_max_dispersive = 0.1 * abs(f_r - f_q)
ax1.axvline(g_max_dispersive, color='red', linestyle='--', alpha=0.7, 
           label=f'Dispersive limit (g = {g_max_dispersive:.0f} MHz)')
ax1.legend()

# Plot quadratic scaling check
chi_01_vs_g = chi_vs_g[:, 1] - chi_vs_g[:, 0]
g_squared_scaled = (g_values / g_values[0])**2 * chi_01_vs_g[0]

ax2.plot(g_values, chi_01_vs_g, 'bo-', label='Calculated χ₁ - χ₀', linewidth=2)
ax2.plot(g_values, g_squared_scaled, 'r--', label='g² scaling', linewidth=2)

ax2.set_xlabel('Coupling Strength g (MHz)')
ax2.set_ylabel('χ₁ - χ₀ (MHz)')
ax2.set_title('Quadratic Scaling Verification')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Coupling Strength Analysis:")
print(f"Dispersive regime limit: g < {g_max_dispersive:.0f} MHz")
for i, (g_val, valid) in enumerate(zip(g_values, dispersive_valid)):
    status = "✓" if valid else "✗"
    print(f"g = {g_val:.0f} MHz: {status} {'Valid' if valid else 'Invalid'} dispersive regime")

## 5. Transmon Energy Level Structure

Let's visualize the energy level structure and understand how it affects the dispersive shifts.

In [ ]:
# Calculate energy levels and transitions
energies = calculate_transmon_energies(f_q, anharmonicity, num_levels)
transitions = calculate_transition_frequencies(f_q, anharmonicity, num_levels)
coupling_matrix = calculate_coupling_matrix_elements(g, num_levels)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Energy level diagram
level_positions = np.arange(num_levels)
ax1.barh(level_positions, energies, alpha=0.7, color='skyblue')
for i, E in enumerate(energies):
    ax1.text(E + 100, i, f'E_{i} = {E:.0f} MHz', va='center')
    
ax1.set_xlabel('Energy (MHz)')
ax1.set_ylabel('Level Number')
ax1.set_title('Transmon Energy Levels')
ax1.grid(True, alpha=0.3)

# Transition frequencies
transition_numbers = np.arange(len(transitions))
bars = ax2.bar(transition_numbers, transitions, alpha=0.7, color='orange')
for i, (bar, freq) in enumerate(zip(bars, transitions)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
             f'{freq:.0f}', ha='center', va='bottom')
    ax2.text(bar.get_x() + bar.get_width()/2, -100, 
             f'|{i}⟩→|{i+1}⟩', ha='center', va='top')

ax2.set_xlabel('Transition')
ax2.set_ylabel('Frequency (MHz)')
ax2.set_title('Transition Frequencies')
ax2.grid(True, alpha=0.3)

# Coupling matrix visualization
im = ax3.imshow(coupling_matrix, cmap='Blues', origin='lower')
ax3.set_xlabel('Final Level')
ax3.set_ylabel('Initial Level')
ax3.set_title('Coupling Matrix Elements |g_nm| (MHz)')

# Add text annotations
for i in range(num_levels):
    for j in range(num_levels):
        if coupling_matrix[i, j] > 0:
            ax3.text(j, i, f'{coupling_matrix[i, j]:.0f}', 
                    ha='center', va='center', color='white', fontweight='bold')

plt.colorbar(im, ax=ax3)
plt.tight_layout()
plt.show()

print("Energy Level Analysis:")
print(f"Level spacings (MHz): {np.diff(energies)}")
print(f"Transition frequencies (MHz): {transitions}")
print(f"Anharmonicity shift per level: {anharmonicity} MHz")
print()
print("Coupling matrix diagonal elements (√n enhancement):")
for i in range(num_levels-1):
    theoretical = g * np.sqrt(i + 1)
    actual = coupling_matrix[i, i+1]
    print(f"g_{i},{i+1} = {actual:.1f} MHz (theory: {theoretical:.1f} MHz)")

## 6. Dispersive Regime Validation

Let's explore the boundaries of the dispersive regime and see what happens when we violate the condition $g \ll |\Delta|$.

In [ ]:
# Test different detunings
detunings = np.logspace(2, 4, 20)  # From 100 MHz to 10 GHz
g_test = 100  # Fixed coupling strength

chi_01_vs_detuning = []
dispersive_ratios = []
theory_predictions = []

for delta in detunings:
    f_r_test = f_q + delta  # Positive detuning
    
    # Calculate chi shifts
    chi = calc.calculate_chi_shifts(f_r_test, f_q, anharmonicity, g_test, 4)
    chi_01 = chi[1] - chi[0]
    chi_01_vs_detuning.append(chi_01)
    
    # Calculate g/Δ ratio
    ratio = g_test / delta
    dispersive_ratios.append(ratio)
    
    # Two-level theory prediction
    theory = g_test**2 / delta
    theory_predictions.append(theory)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Chi vs detuning
ax1.loglog(detunings, np.abs(chi_01_vs_detuning), 'bo-', label='Multi-level calculation', linewidth=2)
ax1.loglog(detunings, theory_predictions, 'r--', label='Two-level theory: g²/Δ', linewidth=2)

ax1.set_xlabel('Detuning |Δ| (MHz)')
ax1.set_ylabel('|χ₁ - χ₀| (MHz)')
ax1.set_title('Chi Shift vs Detuning')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Dispersive regime validity
dispersive_threshold = 0.1
ax2.semilogx(detunings, dispersive_ratios, 'go-', linewidth=2)
ax2.axhline(dispersive_threshold, color='red', linestyle='--', 
           label=f'Dispersive limit (g/Δ = {dispersive_threshold})')
ax2.fill_between(detunings, 0, dispersive_threshold, alpha=0.2, color='red', 
                label='Invalid dispersive regime')
ax2.fill_between(detunings, dispersive_threshold, 1, alpha=0.2, color='green', 
                label='Valid dispersive regime')

ax2.set_xlabel('Detuning |Δ| (MHz)')
ax2.set_ylabel('g/Δ Ratio')
ax2.set_title('Dispersive Regime Validity')
ax2.set_ylim(0, 0.5)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find where dispersive approximation breaks down
relative_errors = np.abs(np.array(chi_01_vs_detuning) - np.array(theory_predictions)) / np.array(theory_predictions)
breakdown_idx = np.where(relative_errors > 0.1)[0]

print("Dispersive Regime Analysis:")
print(f"Coupling strength: {g_test} MHz")
print(f"Dispersive condition: g/Δ < {dispersive_threshold}")
if len(breakdown_idx) > 0:
    breakdown_detuning = detunings[breakdown_idx[0]]
    print(f"Theory breaks down below Δ ≈ {breakdown_detuning:.0f} MHz (g/Δ = {g_test/breakdown_detuning:.3f})")
else:
    print("Theory remains valid for all tested detunings")

## Conclusions

This demonstration shows several key aspects of physics-based chi shift calculations:

1. **Multi-level Effects**: The full calculation accounts for contributions from all virtual transitions, giving more accurate results than the two-level approximation.

2. **Anharmonicity Impact**: More negative anharmonicity increases level spacing, reducing chi shifts for higher levels and enabling better state discrimination.

3. **Coupling Scaling**: Chi shifts scale as $g^2$ in the dispersive regime, but the dispersive condition $g \ll |\Delta|$ must be satisfied.

4. **Level Structure**: The $\sqrt{n}$ enhancement in coupling matrix elements affects how strongly each level couples to the resonator.

5. **Regime Validity**: The dispersive approximation breaks down when $g/|\Delta| \gtrsim 0.1$, requiring more sophisticated treatments.

These insights are crucial for designing optimal readout protocols and understanding the fundamental limits of dispersive qubit measurement.